# Modeling

### What is model exactly??

Model in deep learning is a core component that takes input and output in a certain manner. It is like filter, or function in programming that takes several arguments and output certain values. 

This is a little complicated how to clearly define model and neural network. The relation between them is that neural network is a algorithm and model uses neural network as one of the main components to build. 

- Model is a filter that takes input and output value. 
- Neural network is an algorithm and model uses it as the main component. 

### Transfer Learning

Transfer learning is one of the important methods that eliminate redundant training process in deep learning. Learning process often takes a lot of times and computer power when the training dataset is huge. This is like human teaches to another. 

### Keras Applications

Keras Applications are set of models that is used for transfer learning with pre-trained weights. 

[Keras Applications](https://keras.io/api/applications/)

### MobileNet

MobileNet is a model in deep learning that is invented by Google and effectively designed for computer vision on mobile device. It is a CNN(convolutional neural network) and consumes less computational power to run. 

[MobileNet and MobileNetV2](https://keras.io/api/applications/mobilenet/)

### Development

[Transfer learning with pretrained ConvNet](https://www.tensorflow.org/tutorials/images/transfer_learning)

In [86]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import urllib3
urllib3.disable_warnings()


def mobilenet_model(image_size=160, display_summary=False):
    """Wrapper method of tf.keras.applications.MobileNetV2 to create model.
    
    Arguments
    ---------
    image_size : int
    display_summary : bool
    
    Return
    ------
    model
    """
    IMG_SHAPE = (image_size, image_size, 3)

    # Create the base model from the pre-trained model MobileNet V2.
    # The pre-trained file will be stored in ~/.keras/models/ by default.
    mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet')

    if display_summary:
        mobilenet_model.summary()
        
    return mobilenet_model

def evaluate_model(model, x_test, y_test):
    """Model evaluation
    
    Arguments
    ---------
    model : model
    x_test : numpy.ndarray
        uint8 arrays of digit labels (integers in range 0-9) with shapes (num_samples,).
    y_test : numpy.ndarray
        uint8 arrays of digit labels (integers in range 0-9) with shapes (num_samples,).
        

    References
    ----------
    If you want to know more details about x_test, y_test dataset, 
    please refer to the link.

    https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data?version=nightly#arguments
    """
    
    model.evaluate(x_test, y_test, verbose=2)

# create_mobile_net_model()

In [88]:
if __name__ == "__main__":
    
    model = mobilenet_model()
    model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_21[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________